In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

from keras.preprocessing import text, sequence
from keras import layers, models, optimizers



Using TensorFlow backend.


In [2]:
import json
import os
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from nltk.tag.sequential import ClassifierBasedPOSTagger
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import re
from bs4 import BeautifulSoup
import pickle
from collections import defaultdict
from collections import Counter
from pandas.io.json import json_normalize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()
stem = PorterStemmer()
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer

In [4]:
os.listdir('../input')

['tagging_keyword.json']

In [5]:
with open('../input/tagging_keyword.json') as f:
    d = json.load(f)

In [6]:
file = json_normalize(d)

In [7]:

texts, label = [], []
for k,v in file.items():
    x=eval(v[0])
    s=''
    for i in x:
        s+=i+","
    if(s==''):
        continue
    texts.append(k)
    label.append(s)

In [8]:
tx = set(texts)
lx = set(label)
texts = list(tx)
label = list(lx)
texts = texts[:1709]

In [9]:
trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = label

In [10]:
train_posts = trainDF['text'].unique()
train_tags = trainDF['label'].unique()

test_posts = trainDF['text'].unique()
test_tags = trainDF['label'].unique()

In [11]:
max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

In [12]:
tokenize.fit_on_texts(train_posts) # only fit on train
x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

In [13]:
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [14]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [15]:
# This model trains very quickly and 2 epochs are already more than enough
# Training for more epochs will likely lead to overfitting on this dataset
# You can try tweaking these hyperparamaters when using this model with your own data
batch_size = 32
epochs = 50

In [16]:
# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [17]:
# model.fit trains the model
# The validation_split param tells Keras what % of our training data should be used in the validation set
# You can see the validation loss decreasing slowly when you run this
# Because val_loss is no longer decreasing we stop training to prevent overfitting
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 1538 samples, validate on 171 samples
Epoch 1/50
1538/1538 [==============================] - 1s 938us/step - loss: 7.4689 - acc: 0.0000e+00 - val_loss: 7.5075 - val_acc: 0.0000e+00
Epoch 2/50
1538/1538 [==============================] - 1s 537us/step - loss: 7.2941 - acc: 0.1879 - val_loss: 7.7337 - val_acc: 0.0000e+00
Epoch 3/50
1538/1538 [==============================] - 1s 599us/step - loss: 6.8567 - acc: 0.4857 - val_loss: 8.3694 - val_acc: 0.0000e+00
Epoch 4/50
1538/1538 [==============================] - 1s 626us/step - loss: 6.0796 - acc: 0.6313 - val_loss: 9.1435 - val_acc: 0.0000e+00
Epoch 5/50
1538/1538 [==============================] - 1s 625us/step - loss: 5.0970 - acc: 0.6938 - val_loss: 9.8448 - val_acc: 0.0000e+00
Epoch 6/50
1538/1538 [==============================] - 1s 618us/step - loss: 4.0342 - acc: 0.7289 - val_loss: 10.4105 - val_acc: 0.0000e+00
Epoch 7/50
1538/1538 [==============================] - 1s 614us/step - loss: 3.0478 - acc: 0.7477 - val_los

In [18]:
# Evaluate the accuracy of our trained model
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

1709/1709 [==============================] - 0s 110us/step
Test score: 2.1792652318575634
Test accuracy: 0.7261556465769455


In [19]:
# Here's how to generate a prediction on individual examples
text_labels = encoder.classes_ 

for i in range(100):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_posts[i])
    print('Actual label:' + test_tags[i])
    print("Predicted label: " + predicted_label + "\n")

lessen burden never
Actual label:Life,Poetry,Poet,Written Tales,Poem,
Predicted label: Life,Poetry,Poet,Written Tales,Poem,

teaching yoga house
Actual label:Philosophy,Existentialism,God,Life Lessons,Life,
Predicted label: Philosophy,Existentialism,God,Life Lessons,Life,

lighting good
Actual label:Poetry,Life,Advice,
Predicted label: Poetry,Life,Advice,

compare hair
Actual label:Short Story,Fiction,Fantasy,
Predicted label: Short Story,Fiction,Fantasy,

despair treatment
Actual label:Written Tales,Wt News,Newsletter,
Predicted label: Written Tales,Wt News,Newsletter,

mind context
Actual label:Love,Life Lessons,Relationships,Personal Development,
Predicted label: Love,Life Lessons,Relationships,Personal Development,

probably trouble really
Actual label:Life Lessons,Mental Health,Self Improvement,Startup,Short Story,
Predicted label: Life Lessons,Mental Health,Self Improvement,Startup,Short Story,

quarantined till
Actual label:Poetry,Love,Solitude,
Predicted label: Poetry,Love,Soli

In [20]:
y = pickle.dump(model, open('mdl.pkl','wb'))

In [21]:
mdl = pickle.load(open('mdl.pkl','rb'))


In [22]:
pickle.dump(encoder, open('enco_model.pkl','wb'))
model_enc = pickle.load(open('enco_model.pkl','rb'))

In [23]:
##Creating a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english"))
##Creating a list of custom stopwords
new_words = ["using", "show", "result", "large", "also", "iv", "one", "two", "new", "previously", "shown","how","where","why","it","he","she","of"]
stop_words = stop_words.union(new_words)

In [24]:
st = "What I have realised and you as parents would have also figured is that visuals have a much higher impact on the kids than something which is just written down. So routine charts for kids, or “visual checklists”  are my first line of defence against all variety of annoying behaviour—foot-dragging, defiance, avoidance, ignoring directions, etc."
corpus =[]
#Remove punctuations
soup = BeautifulSoup(st,"lxml")
text = re.sub('[^a-zA-Z]', ' ', soup.get_text())
#Convert to lowercase
text = text.lower()
#remove tags
text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
# remove special characters and digits
text=re.sub("(\\d|\\W)+"," ",text)
#Convert to list from string
text = text.split()
#Stemming
ps=PorterStemmer()
#Lemmatisation
lem = WordNetLemmatizer()
text = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
text = " ".join(text)
corpus.append(text)
if(len(corpus[0])!=0):
    cv=CountVectorizer(max_df=1,min_df=0.95,stop_words=stop_words, max_features=10000, ngram_range=(1,2))
    X=cv.fit_transform(corpus)
    keyword = list(cv.vocabulary_.keys())
else:
    keyword=['']

In [25]:
xtes = tokenize.texts_to_matrix(keyword)
k = model_enc.classes_
arr = []
for i in xtes:
    pred = mdl.predict(np.array([i]))
    arr.append(k[np.argmax(pred)])
t_list = set(arr)
arr = list(t_list)
test_list = Counter(arr) 
res = test_list.most_common(1)[0][0]
print(res)

Artificial Intelligence,Tech,Science Fiction,Fiction,God,
